In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time

# Jax 64-bit
import jax

jax.config.update("jax_enable_x64", True)

In [3]:
from ICARUS.dynamical_systems.first_order_system import NonLinearSystem
from ICARUS.dynamical_systems.second_order_system import SecondOrderSystem
from ICARUS.dynamical_systems.integrate import (
    BackwardEulerIntegrator,
    ForwardEulerIntegrator,
    RK4Integrator,
    RK45Integrator,
    CrankNicolsonIntegrator,
    GaussLegendreIntegrator,
    NewmarkIntegrator,
)

In [4]:
from test_integrators import test_all_integrators

# Simple Mass-Damper System

In [5]:
# Define a simple m-c-k system
m = 1.0
c = 0.1
k = 1.0


def f(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array(
        [
            x[1],  # x' = v
            -c / m * x[1] - k / m * x[0],  # v' = a = -c/m * v - k/m * x
        ]
    )


# Create the system
system = NonLinearSystem(f)

# Test the integrators
test_all_integrators(
    system, jnp.array([1.0, 0.0]), 0.0, 100.0, 0.0001, compare_with_scipy=True
)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 0.4279823303222656 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 1.124176025390625 seconds
Simulating using RK45 integrator
	Simulated using RK45 integrator
	Time taken: 0.6027171611785889 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 4.970059633255005 seconds
Simulating using Gauss-Legendre integrator
	Simulated using Gauss-Legendre integrator
	Time taken: 3.713308095932007 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.12083005905151367 seconds


({'Forward Euler': Array([[ 1.00000000e+00,  0.00000000e+00],
         [ 1.00000000e+00, -1.00000000e-04],
         [ 9.99999990e-01, -1.99999000e-04],
         ...,
         [ 5.16044125e-03,  4.13415918e-03],
         [ 5.16085467e-03,  4.13360180e-03],
         [ 5.16126803e-03,  4.13304438e-03]], dtype=float64),
  'RK4': Array([[ 1.00000000e+00,  0.00000000e+00],
         [ 9.99999995e-01, -9.99994998e-05],
         [ 9.99999980e-01, -1.99997999e-04],
         ...,
         [ 5.13264722e-03,  4.11631063e-03],
         [ 5.13305883e-03,  4.11575619e-03],
         [ 5.13347038e-03,  4.11520170e-03]], dtype=float64),
  'RK45': Array([[ 1.00000000e+00,  0.00000000e+00],
         [ 9.99999995e-01, -9.99994998e-05],
         [ 9.99999978e-01, -2.09997793e-04],
         [ 9.99999945e-01, -3.30994516e-04],
         [ 9.99999892e-01, -4.64089214e-04],
         [ 9.99999814e-01, -6.10491326e-04],
         [ 9.99999702e-01, -7.71531159e-04],
         [ 9.99999550e-01, -9.48671956e-04],
      

# Higher Order

# Second Order Systems

In [6]:
# Define a 2nd order system
m1 = 1.0
c1 = 0.1
k1 = 1.0

m2 = 1.0
c2 = 0.1
k2 = 1.0


def M(t, x):
    return jnp.array([[m1, 0], [0, m2]])


# M = jnp.array([m])
def C(t, x):
    return jnp.array(
        [
            [0.023, 1.024],  # [c1, 0],
            [-0.364, 3.31],  # [0, c2]
        ]
    )


# C = jnp.array([c])


def f_int(t, x):
    return jnp.array(
        [
            [1.97, 0.034],  # [k1, -k1],
            [0.034, 3.95],  # [-k1, k1 + k2]
        ]
    )


# f_int = jnp.array([k])


def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array(
        [
            0.078,  # 0.0,
            10 * 0.466 * jnp.sin(t),
        ]
    )


# f_ext = lambda t, x: jnp.array([0.0])

system = SecondOrderSystem(M, C, f_int, f_ext)

In [7]:
# Test the integrators
test_all_integrators(
    system, jnp.array([0.0, 0.0, 0.0, 0.0]), 0.0, 100.0, 1e-4, compare_with_scipy=True
)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 3.3399362564086914 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 11.663335800170898 seconds
Simulating using RK45 integrator
	Simulated using RK45 integrator
	Time taken: 1.1798315048217773 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 11.234203100204468 seconds
Simulating using Gauss-Legendre integrator
	Simulated using Gauss-Legendre integrator
	Time taken: 0.8328537940979004 seconds
Simulating using Newmark integrator
	Simulated using Newmark integrator
	Time taken: 40.73038578033447 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 0.34417200088500977 seconds


({'Forward Euler': Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  7.80000000e-06,
           4.65999999e-08],
         [ 7.80000000e-10,  4.65999999e-12,  1.55984632e-05,
           1.86355071e-07],
         ...,
         [ 4.71648457e-01, -1.74378855e-01, -6.59411624e-02,
          -1.08097995e+00],
         [ 4.71641863e-01, -1.74486953e-01, -6.58999250e-02,
          -1.08088487e+00],
         [ 4.71635273e-01, -1.74595042e-01, -6.58586851e-02,
          -1.08078975e+00]], dtype=float64),
  'RK4': Array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00],
         [ 3.89974388e-10,  3.10584139e-12,  7.79923164e-06,
           6.98744769e-08],
         [ 1.55979511e-09,  1.78558125e-11,  1.55969266e-05,
           2.32876467e-07],
         ...,
         [ 4.71655369e-01, -1.74389908e-01, -6.59094087e-02,
          -1.08090077e+00],
         [ 4.71648781e-01, -1.74497994e-0